# Usecase

In [645]:
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree
from fuzzywuzzy import process

pd.options.mode.use_inf_as_na = True


/var/folders/pg/n73qslf96w1b52ptxjnqk8q80000gn/T/ipykernel_3385/3481607904.py:6: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  pd.options.mode.use_inf_as_na = True


In [646]:
airbnb_data = pd.read_csv('data/Airbnb_Open_Data.csv')

airbnb_data

/var/folders/pg/n73qslf96w1b52ptxjnqk8q80000gn/T/ipykernel_3385/4067347200.py:1: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  airbnb_data = pd.read_csv('data/Airbnb_Open_Data.csv')


,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,lat,long,country,...,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules,license
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,$193,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...,NaN
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,$28,30.0,45.0,5/21/2022,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...,NaN
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,NaN,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,...,$124,3.0,0.0,NaN,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and...",NaN
3,1002755,NaN,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,...,$74,30.0,270.0,7/5/2019,4.64,4.0,1.0,322.0,NaN,NaN
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,$41,10.0,9.0,11/19/2018,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102594,6092437,Spare room in Williamsburg,12312296767,verified,Krik,Brooklyn,Williamsburg,40.70862,-73.94651,United States,...,$169,1.0,0.0,NaN,NaN,3.0,1.0,227.0,No Smoking No Parties or Events of any kind Pl...,NaN
102595,6092990,Best Location near Columbia U,77864383453,unconfirmed,Mifan,Manhattan,Morningside Heights,40.80460,-73.96545,United States,...,$167,1.0,1.0,7/6/2015,0.02,2.0,2.0,395.0,House rules: Guests agree to the following ter...,NaN
102596,6093542,"Comfy, bright room in Brooklyn",69050334417,unconfirmed,Megan,Brooklyn,Park Slope,40.67505,-73.98045,United States,...,$198,3.0,0.0,NaN,NaN,5.0,1.0,342.0,NaN,NaN
102597,6094094,Big Studio-One Stop from Midtown,11160591270,unconfirmed,Christopher,Queens,Long Island City,40.74989,-73.93777,United States,...,$109,2.0,5.0,10/11/2015,0.10,3.0,1.0,386.0,NaN,NaN


# Data quality check/ Exploratory data analysis

In [647]:
airbnb_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102599 entries, 0 to 102598
Data columns (total 26 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   id                              102599 non-null  int64  
 1   NAME                            102349 non-null  object 
 2   host id                         102599 non-null  int64  
 3   host_identity_verified          102310 non-null  object 
 4   host name                       102193 non-null  object 
 5   neighbourhood group             102570 non-null  object 
 6   neighbourhood                   102583 non-null  object 
 7   lat                             102591 non-null  float64
 8   long                            102591 non-null  float64
 9   country                         102067 non-null  object 
 10  country code                    102468 non-null  object 
 11  instant_bookable                102494 non-null  object 
 12  cancellation_pol

## Entfernen von Duplikaten

In [648]:
airbnb_data = airbnb_data.drop_duplicates()

## Entfernen nicht benötigter Spalten

In [649]:
airbnb_data = airbnb_data.drop(
    ['NAME',
     'id',
     'host id',
     'host_identity_verified',
     'host name',
     'country',
     'country code',
     'instant_bookable',
     'cancellation_policy',
     'Construction year', 
     'minimum nights', 
     'review rate number',
     'calculated host listings count',
     'house_rules',
     'license', 
     'room type'], axis = 1)

airbnb_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 102058 entries, 0 to 102057
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   neighbourhood group  102029 non-null  object 
 1   neighbourhood        102042 non-null  object 
 2   lat                  102050 non-null  float64
 3   long                 102050 non-null  float64
 4   price                101811 non-null  object 
 5   service fee          101785 non-null  object 
 6   number of reviews    101875 non-null  float64
 7   last review          86226 non-null   object 
 8   reviews per month    86240 non-null   float64
 9   availability 365     101610 non-null  float64
dtypes: float64(5), object(5)
memory usage: 8.6+ MB


In [650]:
airbnb_data.describe()

,lat,long,number of reviews,reviews per month,availability 365
count,102050.000000,102050.000000,101875.000000,86240.000000,101610.000000
mean,40.728097,-73.949652,27.517948,1.375408,141.043992
std,0.055852,0.049502,49.571744,1.748019,135.429156
min,40.499790,-74.249840,0.000000,0.010000,-10.000000
25%,40.688730,-73.982580,1.000000,0.220000,3.000000
50%,40.722290,-73.954440,7.000000,0.740000,96.000000
75%,40.762760,-73.932350,31.000000,2.010000,268.000000
max,40.916970,-73.705220,1024.000000,90.000000,3677.000000


## mögliche Werte, die die neighbourhood group Daten annehmen können und berichtigen eventueller Schreibfehler

In [651]:
airbnb_data['neighbourhood group'].unique()

array(['Brooklyn', 'Manhattan', 'brookln', 'manhatan', 'Queens', nan,
       'Staten Island', 'Bronx'], dtype=object)

In [652]:
airbnb_data.loc[airbnb_data['neighbourhood group'] == 'brookln', 'neighbourhood group'] = 'Brooklyn'
airbnb_data.loc[airbnb_data['neighbourhood group'] == 'manhatan', 'neighbourhood group'] = 'Manhattan'

airbnb_data['neighbourhood group'].unique()

array(['Brooklyn', 'Manhattan', 'Queens', nan, 'Staten Island', 'Bronx'],
      dtype=object)

## Prüfen, ob es Schreibfehler, Dopplungen oder ähnliches in neighbourhood gibt

In [653]:
unique_neighbourhoods = airbnb_data['neighbourhood'].dropna().unique()

def find_similar_neighbourhoods(neighbourhoods, threshold=90):
    similar_neighbourhoods = {}
    for neighbourhood in neighbourhoods:
        matches = process.extract(neighbourhood, neighbourhoods, limit=len(neighbourhoods))
        similar = [match for match, score in matches if score >= threshold and match != neighbourhood]
        if similar:
            similar_neighbourhoods[neighbourhood] = similar
    return similar_neighbourhoods

similar_neighbourhoods = find_similar_neighbourhoods(unique_neighbourhoods)

for key, values in similar_neighbourhoods.items():
    print(f"{key}: {values}")

Harlem: ['East Harlem']
East Harlem: ['Harlem']
Upper West Side: ['Upper East Side']
West Village: ['East Village']
Chelsea: ['Chelsea, Staten Island']
East Village: ['West Village']
Flatbush: ['East Flatbush']
Long Island City: ['City Island']
Upper East Side: ['Upper West Side']
Jamaica: ['Jamaica Estates', 'Jamaica Hills']
East Flatbush: ['Flatbush']
Concourse Village: ['Concourse']
Concourse: ['Concourse Village']
Elmhurst: ['East Elmhurst']
Ozone Park: ['South Ozone Park']
East Elmhurst: ['Elmhurst']
Kew Gardens: ['Kew Gardens Hills']
City Island: ['Long Island City']
North Riverdale: ['Riverdale']
Riverdale: ['North Riverdale']
Kew Gardens Hills: ['Kew Gardens']
Bay Terrace: ['Bay Terrace, Staten Island']
Jamaica Estates: ['Jamaica']
South Ozone Park: ['Ozone Park']
East Morrisania: ['Morrisania']
Morrisania: ['East Morrisania']
Holliswood: ['Hollis']
New Dorp Beach: ['New Dorp']
Jamaica Hills: ['Jamaica']
Hollis: ['Holliswood']
Bay Terrace, Staten Island: ['Bay Terrace']
New Dor

## fehlende und ungültige Werte ermitteln und ersetzen

In [654]:
airbnb_data.isna().sum()

neighbourhood group       29
neighbourhood             16
lat                        8
long                       8
price                    247
service fee              273
number of reviews        183
last review            15832
reviews per month      15818
availability 365         448
dtype: int64

### Tabelle der fehlenden Werte für die Standortangaben (neighbourhood group, neighbourhood, lat, long)

In [655]:
missing_data = airbnb_data[
    airbnb_data[['neighbourhood group', 'neighbourhood', 'lat', 'long']].isnull().any(axis=1)
]

missing_data[['neighbourhood group', 'neighbourhood', 'lat', 'long']]

,neighbourhood group,neighbourhood,lat,long
74,NaN,Washington Heights,40.83139,-73.94095
75,NaN,Clinton Hill,40.68346,-73.96374
76,NaN,East Village,40.72828,-73.98801
77,NaN,Upper East Side,40.76865,-73.95058
78,NaN,Woodside,40.75038,-73.90334
90,NaN,Williamsburg,40.71156,-73.96218
91,NaN,Bushwick,40.70032,-73.93830
92,NaN,Prospect Heights,40.68233,-73.97261
148,NaN,East Village,40.72354,-73.98295
161,NaN,Williamsburg,40.71088,-73.95055


#### Prüfen, ob es Datensätze gibt, in denen alle vier Werte fehlen

In [656]:
all_missing_data = airbnb_data[
    airbnb_data[['neighbourhood group', 'neighbourhood', 'lat', 'long']].isnull().all(axis=1)
]

all_missing_data

,neighbourhood group,neighbourhood,lat,long,price,service fee,number of reviews,last review,reviews per month,availability 365


#### Leere neighbourhood group Werte auf Basis der neighbourhood füllen

In [657]:
neighbourhood_group_map = airbnb_data.dropna(subset=['neighbourhood', 'neighbourhood group']).set_index('neighbourhood')['neighbourhood group'].to_dict()

airbnb_data['neighbourhood group'] = airbnb_data.apply(
    lambda row: neighbourhood_group_map[row['neighbourhood']] if pd.isnull(row['neighbourhood group']) and row['neighbourhood'] in neighbourhood_group_map else row['neighbourhood group'],
    axis=1
)

##### ersetzte Werte

In [658]:
missing_data['neighbourhood group'] = missing_data.apply(
    lambda row: neighbourhood_group_map[row['neighbourhood']] if pd.isnull(row['neighbourhood group']) and row['neighbourhood'] in neighbourhood_group_map else row['neighbourhood group'],
    axis=1
)
missing_data[['neighbourhood group', 'neighbourhood', 'lat', 'long']]

/var/folders/pg/n73qslf96w1b52ptxjnqk8q80000gn/T/ipykernel_3385/2549672272.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_data['neighbourhood group'] = missing_data.apply(


,neighbourhood group,neighbourhood,lat,long
74,Manhattan,Washington Heights,40.83139,-73.94095
75,Brooklyn,Clinton Hill,40.68346,-73.96374
76,Manhattan,East Village,40.72828,-73.98801
77,Manhattan,Upper East Side,40.76865,-73.95058
78,Queens,Woodside,40.75038,-73.90334
90,Brooklyn,Williamsburg,40.71156,-73.96218
91,Brooklyn,Bushwick,40.70032,-73.93830
92,Brooklyn,Prospect Heights,40.68233,-73.97261
148,Manhattan,East Village,40.72354,-73.98295
161,Brooklyn,Williamsburg,40.71088,-73.95055


##### noch fehlende Werte

In [659]:
missing_data2 = airbnb_data[
    airbnb_data[['neighbourhood group', 'neighbourhood', 'lat', 'long']].isnull().any(axis=1)
]

missing_data2[['neighbourhood group', 'neighbourhood', 'lat', 'long']]

,neighbourhood group,neighbourhood,lat,long
517,Brooklyn,NaN,40.71580,-73.95803
547,Manhattan,NaN,40.73089,-73.98195
553,Manhattan,NaN,40.71300,-73.99752
575,Manhattan,NaN,40.79816,-73.96190
589,Brooklyn,NaN,40.68012,-73.97847
613,Manhattan,NaN,40.72709,-73.98274
624,Manhattan,NaN,40.75348,-73.97065
633,Manhattan,NaN,40.71693,-73.98948
643,Brooklyn,NaN,40.68016,-73.94878
670,Brooklyn,NaN,40.73641,-73.95330


#### auf Basis von lat und long auf neighbourhood schließen

In [660]:
valid_locations = airbnb_data.dropna(subset=['neighbourhood', 'lat', 'long'])
tree = cKDTree(valid_locations[['lat', 'long']])

def find_nearest_neighbourhood_group(row):
    if pd.isnull(row['neighbourhood']):
        dist, idx = tree.query([row['lat'], row['long']])
        return valid_locations.iloc[idx]['neighbourhood']
    return row['neighbourhood']

airbnb_data['neighbourhood'] = airbnb_data.apply(find_nearest_neighbourhood_group, axis=1)

##### ersetzte Werte

In [661]:
missing_data2['neighbourhood'] = missing_data2.apply(find_nearest_neighbourhood_group, axis=1)
missing_data2[['neighbourhood group', 'neighbourhood', 'lat', 'long']]

/var/folders/pg/n73qslf96w1b52ptxjnqk8q80000gn/T/ipykernel_3385/4273439393.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_data2['neighbourhood'] = missing_data2.apply(find_nearest_neighbourhood_group, axis=1)


,neighbourhood group,neighbourhood,lat,long
517,Brooklyn,Williamsburg,40.71580,-73.95803
547,Manhattan,East Village,40.73089,-73.98195
553,Manhattan,Chinatown,40.71300,-73.99752
575,Manhattan,Upper West Side,40.79816,-73.96190
589,Brooklyn,Park Slope,40.68012,-73.97847
613,Manhattan,East Village,40.72709,-73.98274
624,Manhattan,Midtown,40.75348,-73.97065
633,Manhattan,Lower East Side,40.71693,-73.98948
643,Brooklyn,Bedford-Stuyvesant,40.68016,-73.94878
670,Brooklyn,Greenpoint,40.73641,-73.95330


##### noch fehlende Werte

In [662]:
missing_data3 = airbnb_data[
    airbnb_data[['neighbourhood group', 'neighbourhood', 'lat', 'long']].isnull().any(axis=1)
]

missing_data3[['neighbourhood group', 'neighbourhood', 'lat', 'long']]

,neighbourhood group,neighbourhood,lat,long
779,Brooklyn,Crown Heights,NaN,NaN
785,Brooklyn,Greenpoint,NaN,NaN
799,Manhattan,East Village,NaN,NaN
814,Manhattan,West Village,NaN,NaN
843,Queens,Elmhurst,NaN,NaN
885,Brooklyn,Greenpoint,NaN,NaN
926,Manhattan,Flatiron District,NaN,NaN
986,Manhattan,Upper West Side,NaN,NaN


#### auf Basis von neighbourhood auf lat und long schließen

In [663]:
neighbourhood_coord_map = airbnb_data.dropna(subset=['neighbourhood', 'lat', 'long']).groupby('neighbourhood')[['lat', 'long']].mean().to_dict('index')

def fill_lat_long_from_neighbourhood(row):
    if pd.isnull(row['lat']) or pd.isnull(row['long']):
        if row['neighbourhood'] in neighbourhood_coord_map:
            row['lat'] = neighbourhood_coord_map[row['neighbourhood']]['lat']
            row['long'] = neighbourhood_coord_map[row['neighbourhood']]['long']
    return row

airbnb_data = airbnb_data.apply(fill_lat_long_from_neighbourhood, axis=1)

##### ersetzte Werte

In [664]:
missing_data3 = missing_data3.apply(fill_lat_long_from_neighbourhood, axis=1)

missing_data3[['neighbourhood group', 'neighbourhood', 'lat', 'long']]

,neighbourhood group,neighbourhood,lat,long
779,Brooklyn,Crown Heights,40.672743,-73.945461
785,Brooklyn,Greenpoint,40.727960,-73.950579
799,Manhattan,East Village,40.726750,-73.983855
814,Manhattan,West Village,40.734396,-74.004055
843,Queens,Elmhurst,40.740212,-73.879025
885,Brooklyn,Greenpoint,40.727960,-73.950579
926,Manhattan,Flatiron District,40.741085,-73.988322
986,Manhattan,Upper West Side,40.788932,-73.973239


##### noch fehlende Werte

In [665]:
missing_data4 = airbnb_data[
    airbnb_data[['neighbourhood group', 'neighbourhood', 'lat', 'long']].isnull().any(axis=1)
]

missing_data4[['neighbourhood group', 'neighbourhood', 'lat', 'long']]

,neighbourhood group,neighbourhood,lat,long


In [666]:
airbnb_data.isna().sum()

neighbourhood group        0
neighbourhood              0
lat                        0
long                       0
price                    247
service fee              273
number of reviews        183
last review            15832
reviews per month      15818
availability 365         448
dtype: int64

### Price und service fee als float und fehlende Werte als Durchschnitt der neighbourhood angeben

#### Price und service fee als float

In [667]:
airbnb_data['price'] = airbnb_data['price'].str.replace('$','',  regex= False)
airbnb_data['service fee'] = airbnb_data['service fee'].str.replace('$','',  regex= False)

airbnb_data['price'] = airbnb_data['price'].str.replace(',','',  regex= False)
airbnb_data['service fee'] = airbnb_data['service fee'].str.replace(',','',  regex= False)

#Change all mismatched data types to the appropriate once.
airbnb_data['price'] = airbnb_data['price'].astype(float)
airbnb_data['service fee'] = airbnb_data['service fee'].astype(float)

#### fehlende Wert von price und service fee als Durchschnitt der neighbourhood angeben

In [668]:
airbnb_data['price'] = airbnb_data['price'].fillna(airbnb_data.groupby("neighbourhood")['price'].transform("mean"))

airbnb_data['service fee'] = airbnb_data['service fee'].fillna(airbnb_data.groupby("neighbourhood")['service fee'].transform("mean"))


In [669]:
airbnb_data.isna().sum()

neighbourhood group        0
neighbourhood              0
lat                        0
long                       0
price                      0
service fee                0
number of reviews        183
last review            15832
reviews per month      15818
availability 365         448
dtype: int64

### NaN Werte bei number of reviews als 0 angeben und Datentyp int für die Spalte setzen

In [670]:
airbnb_data['number of reviews'] = airbnb_data['number of reviews'].fillna(0).astype(int)


In [671]:
airbnb_data.isna().sum()

neighbourhood group        0
neighbourhood              0
lat                        0
long                       0
price                      0
service fee                0
number of reviews          0
last review            15832
reviews per month      15818
availability 365         448
dtype: int64

### availability 365 Werte auf Gültigkeit untersuchen und fehlende availability 365 Werte als Durchschnitt der neighbourhood angeben

In [672]:
airbnb_data['availability 365'] = airbnb_data['availability 365'].apply(lambda x: np.nan if x < 0 or x > 365 else x)

In [673]:
airbnb_data['availability 365'] = airbnb_data['availability 365'].fillna(airbnb_data.groupby("neighbourhood")['availability 365'].transform("mean"))


In [674]:
airbnb_data.isna().sum()

neighbourhood group        0
neighbourhood              0
lat                        0
long                       0
price                      0
service fee                0
number of reviews          0
last review            15832
reviews per month      15818
availability 365           0
dtype: int64

### last review und reviews per month bleiben erstmal unbehandelt -> Relevanz ist noch nicht ganz klar

## Statistische Daten und Info nach Abschluss der Analyse und Bereinigung

In [675]:
airbnb_data.describe()

,lat,long,price,service fee,number of reviews,reviews per month,availability 365
count,102058.000000,102058.000000,102058.000000,102058.000000,102058.000000,86240.000000,102058.000000
mean,40.728098,-73.949653,625.355604,125.039943,27.468606,1.375408,134.284838
std,0.055851,0.049501,331.272136,66.237438,49.540962,1.748019,127.569554
min,40.499790,-74.249840,50.000000,10.000000,0.000000,0.010000,0.000000
25%,40.688730,-73.982580,341.000000,68.000000,1.000000,0.220000,4.000000
50%,40.722290,-73.954440,625.000000,125.000000,7.000000,0.740000,100.000000
75%,40.762760,-73.932350,912.000000,182.000000,30.000000,2.010000,248.000000
max,40.916970,-73.705220,1200.000000,240.000000,1024.000000,90.000000,365.000000


In [676]:
airbnb_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 102058 entries, 0 to 102057
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   neighbourhood group  102058 non-null  object 
 1   neighbourhood        102058 non-null  object 
 2   lat                  102058 non-null  float64
 3   long                 102058 non-null  float64
 4   price                102058 non-null  float64
 5   service fee          102058 non-null  float64
 6   number of reviews    102058 non-null  int64  
 7   last review          86226 non-null   object 
 8   reviews per month    86240 non-null   float64
 9   availability 365     102058 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 8.6+ MB
